In [70]:
import pandas as pd
import math
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
from sklearn import metrics
from pytorch_metric_learning import losses
import random
import warnings
warnings.filterwarnings("ignore")

In [56]:
X=pd.read_excel('cms/'+str(1)+'_encoded_train_cms.xlsx').drop(columns=['Unnamed: 0'])
val=pd.read_excel('cms/'+str(1)+'_encoded_validation_cms.xlsx').drop(columns=['Unnamed: 0'])
vlabel=pd.read_csv('cms/'+str(1)+'_cmsvalidationlabel.csv')['0']
y=pd.read_csv('cms/'+str(1)+'_cmstrainlabel.csv')['0']
testlabel=pd.read_csv('cms/'+str(1)+'_cmstestlabel.csv')['0']
test=pd.read_excel('cms/'+str(1)+'_encoded_test_cms.xlsx').reindex(columns=X.columns)
print(X)

      outputscore  isVarchar_1  isDate_1  isInt2_1  isInt4_1  isNumeric_1  \
0        0.076672            0         0         0         1            0   
1        0.017240            0         0         0         1            0   
2        0.156890            0         0         0         1            0   
3        0.355420            0         0         0         1            0   
4        0.354546            0         0         0         1            0   
...           ...          ...       ...       ...       ...          ...   
1417     0.003423            1         0         0         0            0   
1418     0.035458            1         0         0         0            0   
1419     0.016216            0         0         0         1            0   
1420     0.042210            1         0         0         0            0   
1421     0.016216            0         0         0         1            0   

      isOther1  size_1  size_2  isVarchar_2  isDate_2  isInt2_2  isInt4_2  

In [62]:
X=pd.read_excel('mimic/'+str(2)+'_mimictrainEncoded.xlsx').drop(columns=['Unnamed: 0'])
val=pd.read_excel('mimic/'+str(2)+'_mimicvalidationEncoded.xlsx').reindex(columns=X.columns)
vlabel=pd.read_csv('mimic/'+str(2)+'_mimicvalidationlabel.csv')['0']
y=pd.read_csv('mimic/'+str(2)+'_mimictrainlabel.csv')['0']
testlabel=pd.read_csv('mimic/'+str(2)+'_mimictestlabel.csv')['0']
test=pd.read_excel('mimic/'+str(2)+'_mimictestEncoded.xlsx').reindex(columns=X.columns)
print(X)

       outputscore  isVarchar_1  isDate_1  isInt2_1  isInt4_1  isNumeric_1  \
0         0.014911            0         0         0         1            0   
1         0.007701            0         0         0         1            0   
2         0.025479            0         0         0         1            0   
3         0.016071            0         0         0         1            0   
4         0.017303            0         0         0         1            0   
...            ...          ...       ...       ...       ...          ...   
39811     0.003612            1         0         0         0            0   
39812     0.003148            1         0         0         0            0   
39813     0.000213            0         0         0         1            0   
39814     0.002696            1         0         0         0            0   
39815     0.000213            0         0         0         1            0   

       isOther1  size_1  size_2  isVarchar_2  isDate_2  isInt2_

In [155]:
X=pd.read_csv('thalia/unseen_split/'+str(0)+'train_encode.csv').drop(columns=['Unnamed: 0'])
val=pd.read_csv('thalia/unseen_split/'+str(0)+'val_encode.csv').reindex(columns=X.columns)
vlabel=pd.read_csv('thalia/unseen_split/'+str(0)+'val_label.csv')['label']
y=pd.read_csv('thalia/unseen_split/'+str(0)+'train_label.csv')['label']
testlabel=pd.read_csv('thalia/unseen_split/'+str(0)+'test_label.csv')['label']
test=pd.read_csv('thalia/unseen_split/'+str(0)+'test_encode.csv').reindex(columns=X.columns)

In [136]:
X=pd.read_csv('imbd/unseen_split/'+str(3)+'_imdb_train_encode.csv')
X=X.loc[:,~X.columns.str.match("Unnamed")]
val=pd.read_csv('imbd/unseen_split/'+str(3)+'_imdb_val_encode.csv').reindex(columns=X.columns)
vlabel=pd.read_csv('imbd/unseen_split/'+str(3)+'_imdb_val_label.csv')['label']
y=pd.read_csv('imbd/unseen_split/'+str(3)+'_imdb_train_label.csv')['label']
testlabel=pd.read_csv('imbd/unseen_split/'+str(3)+'_imdb_test_label.csv')['label']
test=pd.read_csv('imbd/unseen_split/'+str(3)+'_imdb_test_encode.csv').reindex(columns=X.columns)
X

,primary key_x,foreign key_x,size_x,primary key_y,foreign key_y,size_y,YYYY_x,array_x,boolean_x,integer_x,string_x,array_y,boolean_y,float_y,integer_y,string_y,timestamp_y,outputscore
0,0,0,13,0,1,20,1,0,0,0,0,0,0,0,1,0,0,0.005281
1,0,0,13,0,1,200,1,0,0,0,0,0,0,0,1,0,0,0.004012
2,0,0,13,0,0,15,1,0,0,0,0,0,0,0,0,1,0,0.001581
3,0,0,13,0,0,13,1,0,0,0,0,0,0,0,0,0,1,0.000055
4,0,0,13,0,0,13,1,0,0,0,0,0,0,0,0,0,1,0.000023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1744,1,1,30,1,0,110,0,0,0,0,1,0,0,0,1,0,0,0.203724
1745,1,1,30,0,0,5,0,0,0,0,1,0,0,1,0,0,0,0.004194
1746,1,1,30,0,0,13,0,0,0,0,1,0,0,0,0,0,1,0.000132
1747,1,1,30,0,0,13,0,0,0,0,1,0,0,0,0,0,1,0.002289


In [137]:
## Double check your datasets carefully, if exist discrpency, fix them.
test = test.fillna(0)
val=val.fillna(0)

In [138]:
class Dataset(Dataset):
    def __init__(self, x, y):
#         reader = pd.read_excel(fname)
        all_data = []
        for i in range(x.shape[0]):
            feature = torch.tensor(x.iloc[i].to_list(),dtype=torch.float32).to('cpu')
            class_n = torch.tensor(y[i],dtype=torch.long).to('cpu')
            data = {
                'features': feature,
                'class_n': int(class_n),
            }
            all_data.append(data)
        self.data = all_data
        print('finished data prepration')

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [139]:
train = X
val = val
test = test

"""Here you need input your real label, the lables shown here are just test the model"""
train_y = np.array(y)
val_y =  np.array(vlabel)
test_y = np.array(testlabel)


train = Dataset(train, train_y)
val = Dataset(val, val_y)
test = Dataset(test, test_y)

# Load training dataset labels
temp = [t['class_n'] for t in train]
# Count different classes sample number
class_sample_count = np.array([len(np.where(temp == t)[0]) for t in np.unique(temp)])
# Calculate weights for different classes
weight_temp = 1. / class_sample_count
# Apply weights to each sample
samples_weight = np.array([weight_temp[t] for t in temp])
samples_weight = torch.from_numpy(samples_weight)
# Build sampler for dataloader
sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

train_x = DataLoader(dataset=train, batch_size=64, sampler = sampler)
val_x = DataLoader(dataset=val, batch_size=64, shuffle=False)
test_x = DataLoader(dataset=test, batch_size=64, shuffle=False)

finished data prepration
finished data prepration
finished data prepration


In [140]:
def _reset_params(model, initializer):
    for child in model.children():
        for p in child.parameters():
            if p.requires_grad:
                if initializer == 'uniform':
                    stdv = 1. / math.sqrt(p.shape[0])
                    torch.nn.init.uniform_(p, a=-stdv, b=stdv)
                elif initializer == 'normal':
                    torch.nn.init.normal_(p)
                elif initializer == 'dirac':
                    torch.nn.init.dirac_(p, groups=1)
                elif initializer == 'orthogonal':
                    torch.nn.init.orthogonal_(p,gain=1)
                elif initializer =='kaiming':
                    torch.nn.init.kaiming_uniform_(p,a=0,mode='fan_in',nonlinearity='leaky_relu')
                elif initializer == 'ones':
                    torch.nn.init.ones_(p)
                elif initializer=='xavier_normal':
                    torch.nn.init.xavier_normal_(p,gain=1.0)
                elif initializer =='xavier_uniform':
                    if len(p.shape)>1:
                        torch.nn.init.xavier_uniform_(p)
                    else:
                        stdv = 1. / math.sqrt(p.shape[0])
                        torch.nn.init.uniform_(p, a=-stdv, b=stdv)


def _evaluate_acc_f1(data_loader, model):
    n_correct, n_total = 0, 0
    t_targets_all, t_outputs_all = None, None
    # switch model to evaluation mode
    model.eval()
    with torch.no_grad():
        for t_batch, t_sample_batched in enumerate(data_loader):
            t_inputs = t_sample_batched['features']
            t_targets = t_sample_batched['class_n']
            t_outputs = model(t_inputs)
            n_correct += (torch.argmax(t_outputs, -1) == t_targets).sum().item()
            n_total += len(t_outputs)

            if t_targets_all is None:
                t_targets_all = t_targets
                t_outputs_all = t_outputs
            else:
                t_targets_all = torch.cat((t_targets_all, t_targets), dim=0)
                t_outputs_all = torch.cat((t_outputs_all, t_outputs), dim=0)
    
    acc = n_correct / n_total
    f1 = metrics.f1_score(t_targets_all.cpu(), torch.argmax(t_outputs_all, -1).cpu(), labels=[0, 1], average='macro')
    report = classification_report(t_targets_all.cpu(), torch.argmax(t_outputs_all, -1).cpu(), target_names=['class0','class1'], output_dict=True, digits = 4)
    return acc, f1, report

In [155]:
seed = 1253
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
optimizers = {
    'adadelta': torch.optim.Adadelta,  # default lr=1.0
    'adagrad': torch.optim.Adagrad,  # default lr=0.01
    'adam': torch.optim.Adam,  # default lr=0.001
    'adamax': torch.optim.Adamax,  # default lr=0.002
    'asgd': torch.optim.ASGD,  # default lr=0.01
    'rmsprop': torch.optim.RMSprop,  # default lr=0.01
    'sgd': torch.optim.SGD,
    'adamw':torch.optim.AdamW,
}
"""You should test different Hidden dimension, and learning rate to find the best model"""
N, D_in, H, D_out = 64, 18, 30, 2
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
learning_rate=0.7
_params = filter(lambda p: p.requires_grad, model.parameters())
opt_rec = 'adadelta'
optimizer = optimizers[opt_rec](_params, lr=learning_rate, weight_decay=0.01)
#===================================================================================#
num_epoch = 60
loss_fn = torch.nn.CrossEntropyLoss()
lrshrink = 5

global_step = 0
max_val_f1 = 0
n_correct, n_total, loss_total = 0, 0, 0
t_targets_all, t_outputs_all = None, None
#===================================================================================#
_reset_params(model, 'uniform')
stop_training = False
adam_stop = False
mimicloss=[]
epoch = 0

while not stop_training and epoch <= num_epoch:
    model.train()
    for i_batch, sample_batched in enumerate(train_x):        
        outputs = model(sample_batched['features'])
        targets = sample_batched['class_n']
        loss = loss_fn(outputs, targets)     
        model.zero_grad()
        loss.backward()
        optimizer.step()
        if t_targets_all is None:
            t_targets_all = targets
            t_outputs_all = outputs
        else:
            t_targets_all = torch.cat((t_targets_all, targets), dim=0)
            t_outputs_all = torch.cat((t_outputs_all, outputs), dim=0)

#         report = classification_report(t_targets_all.cpu(), torch.argmax(t_outputs_all, -1).cpu(), target_names=['class0','class1'], output_dict=True, digits = 4)

        n_correct += (torch.argmax(outputs, -1) == targets).sum().item()
        n_total += len(outputs)
        loss_total += loss.item() * len(outputs)
        if global_step % 2 == 0:
            train_acc = n_correct / n_total
            train_loss = loss_total / n_total
            mimicloss.append(train_loss)
            print('loss: {:.4f}, acc: {:.4f}'.format(train_loss, train_acc))

    val_acc, val_f1, report_val = _evaluate_acc_f1(val_x, model)
    print('val_report: {}'.format(report_val['class1']))
    epoch +=1


    if report_val['class1']['f1-score'] > max_val_f1:
        max_val_f1 = report_val['class1']['f1-score']
        print('\nSaving model')
        if not os.path.exists('state_dict/'+'imdb'):
            os.mkdir('state_dict/'+'imdb')
        path = 'state_dict/'+'imdb'+'/{0}_{1}'.format('neuralNet', 'imdb')
        torch.save(model.state_dict(), path)
    else:
        if 'sgd' in opt_rec:
            optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] / lrshrink
            print('Shrinking lr by : {0}. New lr = {1}'
                  .format(lrshrink,
                          optimizer.param_groups[0]['lr']))
            if optimizer.param_groups[0]['lr'] < 1e-5:
                stop_training = True
        if 'adam' in opt_rec:
                    # early stopping (at 2nd decrease in accuracy)
            stop_training = adam_stop
            adam_stop = True



#predict on test
best_path = path

model.load_state_dict(torch.load(best_path,map_location=torch.device('cpu')))
model.eval()
test_acc, test_f1, report_test  = _evaluate_acc_f1(test_x, model)
print(report_test['class1'])

loss: 151.3744, acc: 0.5156
loss: 132.9805, acc: 0.5234
loss: 129.0620, acc: 0.5156
loss: 130.9904, acc: 0.5117
loss: 131.2986, acc: 0.4781
loss: 124.0195, acc: 0.4870
loss: 117.1708, acc: 0.5045
loss: 115.5618, acc: 0.4980
loss: 111.2968, acc: 0.5069
loss: 107.6440, acc: 0.5109
loss: 103.9382, acc: 0.5128
loss: 100.0029, acc: 0.5091
loss: 96.7806, acc: 0.5024
loss: 92.1061, acc: 0.5078
loss: 88.9169, acc: 0.5062
loss: 85.4714, acc: 0.4990
loss: 82.0202, acc: 0.4963
loss: 78.3920, acc: 0.4878
loss: 74.4904, acc: 0.4827
loss: 70.8419, acc: 0.4922
loss: 67.5727, acc: 0.4985
loss: 64.6231, acc: 0.4972
loss: 61.9716, acc: 0.4986
loss: 59.4519, acc: 0.5078
loss: 57.1446, acc: 0.5131
loss: 55.0062, acc: 0.5108
loss: 53.0591, acc: 0.5133
loss: 52.4576, acc: 0.5129
val_report: {'precision': 0.024793388429752067, 'recall': 0.75, 'f1-score': 0.048, 'support': 4}

Saving model
loss: 50.7541, acc: 0.5174
loss: 49.1361, acc: 0.5173
loss: 47.6135, acc: 0.5245
loss: 46.1523, acc: 0.5247
loss: 44.7830

loss: 7.7274, acc: 0.6795
loss: 7.6999, acc: 0.6803
loss: 7.6811, acc: 0.6808
loss: 7.6585, acc: 0.6803
loss: 7.6352, acc: 0.6809
loss: 7.6098, acc: 0.6814
loss: 7.5836, acc: 0.6814
loss: 7.5591, acc: 0.6818
loss: 7.5326, acc: 0.6824
loss: 7.5239, acc: 0.6826
val_report: {'precision': 0.42857142857142855, 'recall': 0.75, 'f1-score': 0.5454545454545454, 'support': 4}
loss: 7.4976, acc: 0.6835
loss: 7.4714, acc: 0.6842
loss: 7.4457, acc: 0.6847
loss: 7.4209, acc: 0.6851
loss: 7.4203, acc: 0.6845
loss: 7.3965, acc: 0.6848
loss: 7.3725, acc: 0.6851
loss: 7.3502, acc: 0.6848
loss: 7.3293, acc: 0.6851
loss: 7.3069, acc: 0.6856
loss: 7.2827, acc: 0.6862
loss: 7.2584, acc: 0.6867
loss: 7.2377, acc: 0.6870
loss: 7.2139, acc: 0.6876
loss: 7.1944, acc: 0.6878
loss: 7.1855, acc: 0.6875
loss: 7.1623, acc: 0.6881
loss: 7.1387, acc: 0.6884
loss: 7.1152, acc: 0.6889
loss: 7.0927, acc: 0.6889
loss: 7.0744, acc: 0.6893
loss: 7.0512, acc: 0.6901
loss: 7.0281, acc: 0.6907
loss: 7.0054, acc: 0.6914
loss: 6

loss: 3.8056, acc: 0.7735
loss: 3.7999, acc: 0.7738
loss: 3.7943, acc: 0.7741
loss: 3.7891, acc: 0.7741
loss: 3.7858, acc: 0.7743
loss: 3.7816, acc: 0.7744
loss: 3.7832, acc: 0.7741
loss: 3.7780, acc: 0.7744
loss: 3.7737, acc: 0.7745
loss: 3.7695, acc: 0.7747
loss: 3.7678, acc: 0.7747
val_report: {'precision': 0.0625, 'recall': 0.75, 'f1-score': 0.11538461538461539, 'support': 4}
loss: 3.7636, acc: 0.7748
loss: 3.7583, acc: 0.7750
loss: 3.7528, acc: 0.7753
loss: 3.7475, acc: 0.7755
loss: 3.7453, acc: 0.7753
loss: 3.7422, acc: 0.7754
loss: 3.7378, acc: 0.7755
loss: 3.7327, acc: 0.7757
loss: 3.7277, acc: 0.7758
loss: 3.7235, acc: 0.7760
loss: 3.7187, acc: 0.7762
loss: 3.7133, acc: 0.7765
loss: 3.7080, acc: 0.7768
loss: 3.7027, acc: 0.7771
loss: 3.6974, acc: 0.7773
loss: 3.6921, acc: 0.7777
loss: 3.6871, acc: 0.7778
loss: 3.6829, acc: 0.7777
loss: 3.6796, acc: 0.7779
loss: 3.6758, acc: 0.7781
loss: 3.6708, acc: 0.7782
loss: 3.6685, acc: 0.7783
loss: 3.6655, acc: 0.7782
loss: 3.6617, acc: 

loss: 2.8174, acc: 0.8167
loss: 2.8155, acc: 0.8164
loss: 2.8166, acc: 0.8165
loss: 2.8138, acc: 0.8167
loss: 2.8110, acc: 0.8168
loss: 2.8082, acc: 0.8170
loss: 2.8054, acc: 0.8172
loss: 2.8026, acc: 0.8174
loss: 2.7999, acc: 0.8175
loss: 2.7972, acc: 0.8175
loss: 2.7951, acc: 0.8176
loss: 2.7923, acc: 0.8178
loss: 2.7895, acc: 0.8179
loss: 2.7867, acc: 0.8181
loss: 2.7840, acc: 0.8183
loss: 2.7812, acc: 0.8184
loss: 2.7785, acc: 0.8186
loss: 2.7760, acc: 0.8187
loss: 2.7737, acc: 0.8185
loss: 2.7732, acc: 0.8185
loss: 2.7708, acc: 0.8186
loss: 2.7685, acc: 0.8185
loss: 2.7672, acc: 0.8186
loss: 2.7649, acc: 0.8187
loss: 2.7624, acc: 0.8188
loss: 2.7635, acc: 0.8186
loss: 2.7621, acc: 0.8186
loss: 2.7612, acc: 0.8187
val_report: {'precision': 0.18181818181818182, 'recall': 1.0, 'f1-score': 0.3076923076923077, 'support': 4}
loss: 2.7586, acc: 0.8187
loss: 2.7562, acc: 0.8187
loss: 2.7538, acc: 0.8189
loss: 2.7514, acc: 0.8190
loss: 2.7494, acc: 0.8191
loss: 2.7491, acc: 0.8189
loss: 2.

loss: 2.2898, acc: 0.8447
loss: 2.2884, acc: 0.8446
loss: 2.2885, acc: 0.8446
loss: 2.2868, acc: 0.8447
loss: 2.2850, acc: 0.8448
loss: 2.2833, acc: 0.8450
loss: 2.2815, acc: 0.8451
loss: 2.2798, acc: 0.8452
loss: 2.2780, acc: 0.8453
loss: 2.2763, acc: 0.8454
loss: 2.2745, acc: 0.8455
loss: 2.2728, acc: 0.8456
loss: 2.2711, acc: 0.8458
loss: 2.2694, acc: 0.8459
loss: 2.2676, acc: 0.8460
loss: 2.2659, acc: 0.8461
loss: 2.2642, acc: 0.8462
loss: 2.2626, acc: 0.8463
loss: 2.2633, acc: 0.8461
loss: 2.2625, acc: 0.8462
loss: 2.2612, acc: 0.8462
loss: 2.2606, acc: 0.8461
loss: 2.2597, acc: 0.8461
loss: 2.2587, acc: 0.8461
loss: 2.2574, acc: 0.8461
loss: 2.2558, acc: 0.8462
loss: 2.2559, acc: 0.8462
val_report: {'precision': 0.5, 'recall': 0.75, 'f1-score': 0.6, 'support': 4}
loss: 2.2583, acc: 0.8460
loss: 2.2585, acc: 0.8459
loss: 2.2575, acc: 0.8458
loss: 2.2574, acc: 0.8458
loss: 2.2559, acc: 0.8459
loss: 2.2543, acc: 0.8460
loss: 2.2529, acc: 0.8459
loss: 2.2522, acc: 0.8459
loss: 2.2505

loss: 1.9394, acc: 0.8646
loss: 1.9382, acc: 0.8647
loss: 1.9370, acc: 0.8647
loss: 1.9358, acc: 0.8648
loss: 1.9346, acc: 0.8649
loss: 1.9334, acc: 0.8650
loss: 1.9322, acc: 0.8651
loss: 1.9310, acc: 0.8651
loss: 1.9298, acc: 0.8652
loss: 1.9286, acc: 0.8653
loss: 1.9274, acc: 0.8654
loss: 1.9270, acc: 0.8654
val_report: {'precision': 0.4444444444444444, 'recall': 1.0, 'f1-score': 0.6153846153846153, 'support': 4}
loss: 1.9258, acc: 0.8655
loss: 1.9246, acc: 0.8656
loss: 1.9234, acc: 0.8657
loss: 1.9222, acc: 0.8658
loss: 1.9211, acc: 0.8658
loss: 1.9204, acc: 0.8659
loss: 1.9201, acc: 0.8658
loss: 1.9196, acc: 0.8658
loss: 1.9184, acc: 0.8659
loss: 1.9172, acc: 0.8660
loss: 1.9162, acc: 0.8660
loss: 1.9166, acc: 0.8659
loss: 1.9166, acc: 0.8659
loss: 1.9155, acc: 0.8659
loss: 1.9143, acc: 0.8660
loss: 1.9132, acc: 0.8661
loss: 1.9121, acc: 0.8661
loss: 1.9112, acc: 0.8661
loss: 1.9101, acc: 0.8662
loss: 1.9093, acc: 0.8662
loss: 1.9100, acc: 0.8660
loss: 1.9097, acc: 0.8660
loss: 1.9